In [86]:
import sounddevice as sd
import librosa
import numpy as np
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler



In [87]:

def extract_features(signal, sr):
    features = {}
    
    # Fundamental frequency features
    pitches = librosa.yin(signal, fmin=50, fmax=300, sr=sr)
    features['MDVP:Fo(Hz)'] = pitches.mean()
    features['MDVP:Fhi(Hz)'] = pitches.max()
    features['MDVP:Flo(Hz)'] = pitches.min()
    
    # Jitter features
    features['MDVP:Jitter(%)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Jitter(Abs)'] = np.mean(np.abs(signal))
    features['MDVP:RAP'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:PPQ'] = np.sqrt(np.mean(signal ** 2))
    features['Jitter:DDP'] = features['MDVP:RAP'] * 3
    
    # Shimmer features
    features['MDVP:Shimmer'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Shimmer(dB)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:APQ'] = np.sqrt(np.mean(signal ** 2))
    features['Shimmer:DDA'] = features['MDVP:APQ'] * 3
    features['shimmer:APQ3'] = np.sqrt(np.mean(signal ** 2))
    features['shimmer:APQ5'] = np.sqrt(np.mean(signal ** 2))

    # Other features
    features['NHR'] = np.sqrt(np.mean(signal ** 2))
    features['HNR'] = np.sqrt(np.mean(signal ** 2))
    features['RPDE'] = np.sqrt(np.mean(signal ** 2))
    features['DFA'] = np.sqrt(np.mean(signal ** 2))
    features['spread1'] = -np.sqrt(np.mean(signal ** 2))
    features['spread2'] = np.sqrt(np.mean(signal ** 2))
    features['D2'] = np.sqrt(np.mean(signal ** 2))
    features['PPE'] = np.sqrt(np.mean(signal ** 2))
    
    return features


In [89]:
# Set the sample rate for recording and processing the audio
sample_rate = 22050  # Sample rate in Hz
duration = 10  # Duration in seconds


In [90]:
# Record audio from the microphone
print('Recording audio...')
recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
sd.wait()  # Wait for the recording to complete


Recording audio...


In [91]:
# Flatten the recording array to one-dimensional
signal = recording.flatten()


In [92]:
# Extract features from the recorded audio
extracted_features = extract_features(signal, sample_rate)


In [93]:
# Print the extracted features
for feature, value in extracted_features.items():
    print(f'{feature}: {value}')
    

MDVP:Fo(Hz): 119.18131675290358
MDVP:Fhi(Hz): 302.05479452054794
MDVP:Flo(Hz): 53.84911186339075
MDVP:Jitter(%): 0.0013579943915829062
MDVP:Jitter(Abs): 0.0007893481524661183
MDVP:RAP: 0.0013579943915829062
MDVP:PPQ: 0.0013579943915829062
Jitter:DDP: 0.004073983174748719
MDVP:Shimmer: 0.0013579943915829062
MDVP:Shimmer(dB): 0.0013579943915829062
MDVP:APQ: 0.0013579943915829062
Shimmer:DDA: 0.004073983174748719
shimmer:APQ3: 0.0013579943915829062
shimmer:APQ5: 0.0013579943915829062
NHR: 0.0013579943915829062
HNR: 0.0013579943915829062
RPDE: 0.0013579943915829062
DFA: 0.0013579943915829062
spread1: -0.0013579943915829062
spread2: 0.0013579943915829062
D2: 0.0013579943915829062
PPE: 0.0013579943915829062


In [94]:
# Round off the extracted features up to 5 decimal places
rounded_features = {}
for key, value in extracted_features.items():
    rounded_features[key] = round(value, 5)
    print(rounded_features[key])

119.18132
302.05479
53.84911
0.00136
0.00079
0.00136
0.00136
0.00407
0.00136
0.00136
0.00136
0.00407
0.00136
0.00136
0.00136
0.00136
0.00136
0.00136
-0.00136
0.00136
0.00136
0.00136


In [111]:
# Preprocess the extracted features

feature_vector = []
feature_order = ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP',
                 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'MDVP:APQ', 'Shimmer:DDA','shimmer:APQ3','shimmer:APQ5',
                 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

for feature in feature_order:
    feature_vector.append(rounded_features[feature])

print(feature_vector)

[119.18132, 302.05479, 53.84911, 0.00136, 0.00079, 0.00136, 0.00136, 0.00407, 0.00136, 0.00136, 0.00136, 0.00407, 0.00136, 0.00136, 0.00136, 0.00136, 0.00136, 0.00136, -0.00136, 0.00136, 0.00136, 0.00136]


In [118]:
# Convert the feature vector to a numpy array
feature_vector = np.array(feature_vector).reshape(1, -1)

# Round the feature vector to 5 decimal places precision
rounded_feature_vector = np.round(feature_vector, decimals=5)
# Modify display settings to disable scientific notation
np.set_printoptions(precision=5, suppress=True)

print(rounded_feature_vector)

#print(feature_vector)

[[119.18132 302.05479  53.84911   0.00136   0.00079   0.00136   0.00136
    0.00407   0.00136   0.00136   0.00136   0.00407   0.00136   0.00136
    0.00136   0.00136   0.00136   0.00136  -0.00136   0.00136   0.00136
    0.00136]]


In [114]:
sample_data=[[171.04100,208.31300,75.50100,0.00455,0.00003,0.00250,0.00234,0.00750,0.01966,0.18600,0.00889,0.01169,0.01872,0.02666,0.01095,25.90800,0.418622,0.720916,-6.183590,0.226278,2.589702,0.147403],
          [146.84500,208.70100,81.73700,0.00496,0.00003,0.00250,0.00275,0.00749,0.01919,0.19800,0.00883,0.01144,0.01826,0.02650,0.01328,25.11900,0.358773,0.726652,-6.271690,0.196102,2.314209,0.162999],
          [155.35800,227.38300,80.05500,0.00310,0.00002,0.00159,0.00176,0.00476,0.01718,0.16100,0.00769,0.01012,0.01661,0.02307,0.00677,25.97000,0.470478,0.676258,-7.120925,0.279789,2.241742,0.108514],
          [162.56800,198.34600,77.63000,0.00502,0.00003,0.00280,0.00253,0.00841,0.01791,0.16800,0.00793,0.01057,0.01799,0.02380,0.01170,25.67800,0.427785,0.723797,-6.635729,0.209866,1.957961,0.135242],
          [120.26700,137.24400,114.82000,0.00333,0.00003,0.00155,0.00202,0.00466,0.01608,0.14000,0.00779,0.00937,0.01351,0.02337,0.00607,24.88600,0.596040,0.764112,-5.634322,0.257682,1.854785,0.211756],
          [197.07600,206.89600,192.05500,0.00289,0.00001,0.00166,0.00168,0.00498,0.01098,0.09700,0.00563,0.00680,0.00802,0.01689,0.00339,26.77500,0.422229,0.741367,-7.348300,0.177551,1.743867,0.085569],
          [199.22800,209.51200,192.09100,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000,0.432439,0.742055,-7.682587,0.173319,2.103106,0.068501],
          [198.38300,215.20300,193.10400,0.00212,0.00001,0.00113,0.00135,0.00339,0.01263,0.11100,0.00640,0.00825,0.00951,0.01919,0.00119,30.77500,0.465946,0.738703,-7.067931,0.175181,1.512275,0.096320],
          [202.26600,211.60400,197.07900,0.00180,0.000009,0.00093,0.00107,0.00278,0.00954,0.08500,0.00469,0.00606,0.00719,0.01407,0.00072,32.68400,0.368535,0.742133,-7.695734,0.178540,1.544609,0.056141],
          [203.18400,211.52600,196.16000,0.00178,0.000009,0.00094,0.00106,0.00283,0.00958,0.08500,0.00468,0.00610,0.00726,0.01403,0.00065,33.04700,0.340068,0.741899,-7.964984,0.163519,1.423287,0.044539]
]
new_data =[]
new_data =[feature_vector[0]]
print(new_data)

[array([ 1.2e+02,  3.0e+02,  5.4e+01,  1.4e-03,  7.9e-04,  1.4e-03,
        1.4e-03,  4.1e-03,  1.4e-03,  1.4e-03,  1.4e-03,  4.1e-03,
        1.4e-03,  1.4e-03,  1.4e-03,  1.4e-03,  1.4e-03,  1.4e-03,
       -1.4e-03,  1.4e-03,  1.4e-03,  1.4e-03])]


In [98]:
scaler=StandardScaler()
new_data_scaled=scaler.fit_transform(new_data)
print(new_data_scaled)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [99]:
# Load the pre-trained SVM model
svm_model = joblib.load('C:/Users/hp/Documents/PYTHON/Parkinsons detection/model.pkl')

In [100]:
# Perform prediction using the SVM model
prediction = svm_model.predict(feature_vector)


In [104]:
# Print the prediction
print(f"Prediction: {prediction}")

Prediction: [0]
